In [1]:
import os
import sys
from importlib import reload
from pathlib import Path

import numpy as np
import pandas as pd
from torch.nn import ReLU
from torch_geometric.nn import GCNConv

In [2]:
# NOTE: Boilerplate setup for Jupyter imports
sys.path.append(Path(os.getcwd()).parent.as_posix())

import configs as config_module
import enhancer as enhancer_module
import encoders as encoders_module
import scheme.network as network_module
import scheme.data as data_module
from scheme.network import GNNConfig

reload(config_module)
reload(enhancer_module)
reload(encoders_module)
reload(network_module)
reload(data_module)

PathConfig, TrainConfig = config_module.PathConfig, config_module.TrainConfig
Enhancer = enhancer_module.Enhancer
get_default_encoders = encoders_module.get_default_encoders
GNNConfig, EnhancerData = network_module.GNNConfig, data_module.EnhancerData

In [3]:
path_config = PathConfig(
    data_dir="../../data"
)

with open(path_config.target_data, "rb") as f:
    unpacked = np.load(f)

    # NOTE: Target dimensions
    data = EnhancerData(
        unpacked["data"],
        unpacked["target"].reshape(-1),
        unpacked["spatial"],
    )

In [4]:
gnn_config = GNNConfig(
    activation=ReLU,
    activation_args={},

    conv_operator=GCNConv,
    conv_args={ },

    encoder_scheme=[data.features.shape[1], 256, 256],
    estimator_scheme=[128, 128, np.unique(data.target).shape[0]],
)

In [5]:
encoders = [
    encoders_module.DistEncoder(
        max_dist=5,
        cache_dir=path_config.edge_cache,
        note="cora_dist",
    ),
    encoders_module.ReprEncoder(
        neighbor_rate=0.7,
        cache_dir=path_config.edge_cache,
        note="cora_repr",
    ),
]

result = Enhancer.compare_builders(data, gnn_config, encoders)

cora_dist test: Loss = 1.3343e+00 | Accuracy = 0.7158671586715867
cora_repr test: Loss = 1.5959e+00 | Accuracy = 0.44649446494464945
DistEncoder test: Loss = 1.2298e+00 | Accuracy = 0.6568265682656826
ReprEncoder test: Loss = 1.5955e+00 | Accuracy = 0.43911439114391143


In [6]:
print(result)

Option         accuracy_score    f1_score
-----------  ----------------  ----------
cora_dist            0.715867    0.694208
cora_repr            0.446494    0.349414
DistEncoder          0.656827    0.647604
ReprEncoder          0.439114    0.331533


In [7]:
enhancer = Enhancer(
    net_config=GNNConfig(
        activation=ReLU,
        activation_args={},

        conv_operator=GCNConv,
        conv_args={ },

        encoder_scheme=[data.features.shape[1], 256, 256],
        estimator_scheme=[128, 128, np.unique(data.target).shape[0]],
    ),
    edge_builder=encoders_module.DistEncoder(
        max_dist=5,
        cache_dir=path_config.edge_cache,
        note="cora_dist [max dist = 5]",
    ),
)

In [8]:
gnn = enhancer.fit(data)

In [12]:
enhancer.transform(data)

tensor([[-0.3291,  0.4623, -0.1789,  ...,  0.2966, -0.2652,  0.0371],
        [ 0.1810,  1.6030, -0.5669,  ...,  0.4732, -0.6694,  0.4383],
        [-0.0119,  1.2705, -0.1835,  ...,  0.3604, -0.5643,  0.2236],
        ...,
        [ 0.0488,  1.2603,  0.1931,  ...,  0.1997, -0.6257,  0.1899],
        [-0.2178,  0.7619,  0.0170,  ...,  0.3220, -0.6603, -0.0411],
        [-0.3046,  0.6646,  0.0055,  ...,  0.3031, -0.6487,  0.0435]],
       grad_fn=<AddBackward0>)